In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


df=pd.read_csv("data.txt")

df = pd.get_dummies(df, columns=['team'], dtype = int)
df = pd.get_dummies(df, columns=['quarter'], dtype = int)
df = pd.get_dummies(df, columns=['day'], dtype = int)

df['department'] = df['department'].replace('finishing ', 'finishing')


df.head()

,date,department,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,...,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,quarter_Quarter5,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,1/1/2015,sweing,0.80,26.16,1108.0,7080,98,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1/1/2015,finishing,0.75,3.94,NaN,960,0,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1/1/2015,sweing,0.80,11.41,968.0,3660,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1/1/2015,sweing,0.80,11.41,968.0,3660,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1/1/2015,sweing,0.80,25.90,1170.0,1920,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
df_sinFinishing = df.drop(df[df['department'] == 'finishing'].index)

valores_unicos = df_sinFinishing['department'].unique()
print(valores_unicos)



['sweing']


In [5]:
df_sinFinishing.head()

,date,department,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,...,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,quarter_Quarter5,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,1/1/2015,sweing,0.8,26.16,1108.0,7080,98,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1/1/2015,sweing,0.8,11.41,968.0,3660,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1/1/2015,sweing,0.8,11.41,968.0,3660,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1/1/2015,sweing,0.8,25.90,1170.0,1920,50,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,1/1/2015,sweing,0.8,25.90,984.0,6720,38,0.0,0,0,...,0,0,0,0,0,0,0,1,0,0


Modelo de Regresion Sin WIP

In [7]:
features = ['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
       'team_1', 'team_2', 'team_3', 'team_4', 'team_5',
       'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 
       # 'team_12',
       'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 
       # 'quarter_Quarter5', 
       'day_Monday', 'day_Saturday',
       'day_Sunday', 'day_Thursday', 'day_Tuesday', 
       # 'day_Wednesday',
]

X = df_sinFinishing[features]

X.head()

,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,team_1,...,team_11,quarter_Quarter1,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday
0,0.8,26.16,1108.0,7080,98,0.0,0,0,59.0,0,...,0,1,0,0,0,0,0,0,1,0
2,0.8,11.41,968.0,3660,50,0.0,0,0,30.5,0,...,1,1,0,0,0,0,0,0,1,0
3,0.8,11.41,968.0,3660,50,0.0,0,0,30.5,0,...,0,1,0,0,0,0,0,0,1,0
4,0.8,25.90,1170.0,1920,50,0.0,0,0,56.0,0,...,0,1,0,0,0,0,0,0,1,0
5,0.8,25.90,984.0,6720,38,0.0,0,0,56.0,0,...,0,1,0,0,0,0,0,0,1,0


In [8]:
Y = df_sinFinishing['actual_productivity']

Y.head()

0    0.940725
2    0.800570
3    0.800570
4    0.800382
5    0.800125
Name: actual_productivity, dtype: float64

In [9]:
print(type(X))
print(type(Y))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


Division Entre Entrenamiento y Prueba

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1, test_size=0.3)

In [11]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(483, 29)
(483,)
(208, 29)
(208,)


In [12]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()

linreg.fit(X_train, Y_train)

# print('Intercepto: ' + str(linreg.intercept_))

# print('Coeficientes: ' + str(linreg.coef_))

list(zip(features, linreg.coef_))

[('targeted_productivity', 0.6035693616429564),
 ('smv', -0.004661484379280834),
 ('wip', -1.3167442103703365e-06),
 ('over_time', -4.5188564114910635e-06),
 ('incentive', 0.0032983072827240632),
 ('idle_time', 0.022274846974669728),
 ('idle_men', -0.01069084738653728),
 ('no_of_style_change', 0.0018732740300281458),
 ('no_of_workers', 0.002866687485797949),
 ('team_1', -0.012701548786089912),
 ('team_2', 0.005744946259922142),
 ('team_3', -0.004381189796524392),
 ('team_4', 0.00807121900588684),
 ('team_5', 0.005488280596831818),
 ('team_6', 0.021488049787545132),
 ('team_7', 0.02066157016083977),
 ('team_8', 0.000169608377903331),
 ('team_9', -0.013580697415425027),
 ('team_10', -0.028375332262650082),
 ('team_11', -0.05109367191961332),
 ('quarter_Quarter1', 0.027280977909785505),
 ('quarter_Quarter2', 0.019506102509062352),
 ('quarter_Quarter3', 0.01968826838013819),
 ('quarter_Quarter4', 0.010741937796367023),
 ('day_Monday', 0.0230033218610258),
 ('day_Saturday', 0.01393193265988

# Predicciones Usando los Datos de Prueba

In [13]:
Y_pred = linreg.predict(X_test)

Evaluar el Modelo

In [14]:
from sklearn import metrics

import numpy as np

# mean absolute error
MAE = metrics.mean_absolute_error(Y_test, Y_pred)

# mean squared error
MSE = metrics.mean_squared_error(Y_test, Y_pred)

# root mean squared error
RMSE = np.sqrt(MSE)

print("MAE: ", MAE)
print("MSE: ", MSE)
print("RMSE: ", RMSE)

MAE:  0.12169831366768345
MSE:  0.4167990592162488
RMSE:  0.6455997670509561


Stats Models

In [18]:
import statsmodels.api as sm

features = ['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
       'team_1', 'team_2', 'team_3', 'team_4', 'team_5',
       'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 
       # 'team_12',
       'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 
       # 'quarter_Quarter5', 
       'day_Monday', 'day_Saturday',
       'day_Sunday', 'day_Thursday', 'day_Tuesday', 
       # 'day_Wednesday',
]

X = df_sinFinishing [features]
Y = df_sinFinishing['actual_productivity']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

X_train = sm.add_constant(X_train)

model = sm.OLS(Y_train, X_train).fit()

print(model.summary())


                             OLS Regression Results                            
Dep. Variable:     actual_productivity   R-squared:                       0.803
Model:                             OLS   Adj. R-squared:                  0.792
Method:                  Least Squares   F-statistic:                     68.80
Date:                 Mon, 11 Mar 2024   Prob (F-statistic):          2.22e-152
Time:                         01:34:18   Log-Likelihood:                 645.47
No. Observations:                  518   AIC:                            -1231.
Df Residuals:                      488   BIC:                            -1103.
Df Model:                           29                                         
Covariance Type:             nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const               

In [20]:
summary_table = model.summary().tables[1]
p_values = [row[4] for row in summary_table.data[1:]]
variables = [row[0] for row in summary_table.data[1:]]

p_values = [float(pvalue.strip()) for pvalue in p_values]

for i in range(len(variables)):
    pvalue = (p_values[i])
    variable = variables[i]
    if pvalue < 0.1:
        texto = 'Significativo'
    else:
        texto = 'NO Significativo'
    print(variable + ': ' + texto)


const: Significativo
targeted_productivity: Significativo
smv: Significativo
wip: NO Significativo
over_time: Significativo
incentive: Significativo
idle_time: Significativo
idle_men: Significativo
no_of_style_change: NO Significativo
no_of_workers: Significativo
team_1: NO Significativo
team_2: NO Significativo
team_3: NO Significativo
team_4: NO Significativo
team_5: NO Significativo
team_6: NO Significativo
team_7: NO Significativo
team_8: NO Significativo
team_9: NO Significativo
team_10: NO Significativo
team_11: Significativo
quarter_Quarter1: NO Significativo
quarter_Quarter2: NO Significativo
quarter_Quarter3: NO Significativo
quarter_Quarter4: NO Significativo
day_Monday: NO Significativo
day_Saturday: NO Significativo
day_Sunday: NO Significativo
day_Thursday: NO Significativo
day_Tuesday: NO Significativo


In [21]:

Y_pred = model.predict(sm.add_constant(X_test))

mse_model = mean_squared_error(Y_test, Y_pred)

print("Error cuadrático medio del primer modelo:", mse_model)

Error cuadrático medio del primer modelo: 0.0041733683876170775


In [29]:
new_features = [
    'targeted_productivity', 'smv'
   #  , 'wip'
    , 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
       'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 
    #    'team_1', 'team_2', 'team_3', 'team_4', 'team_5',
    #    'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 
       # 'team_12',

]

X_new = df_sinFinishing[new_features]
Y_new = df_sinFinishing['actual_productivity']

X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(X_new, Y_new, random_state=1)

X_train_new = sm.add_constant(X_train_new)

model_new = sm.OLS(Y_train_new, X_train_new).fit()

print(model_new.summary())

                             OLS Regression Results                            
Dep. Variable:     actual_productivity   R-squared:                       0.790
Model:                             OLS   Adj. R-squared:                  0.785
Method:                  Least Squares   F-statistic:                     158.7
Date:                 Mon, 11 Mar 2024   Prob (F-statistic):          1.21e-162
Time:                         01:42:57   Log-Likelihood:                 628.81
No. Observations:                  518   AIC:                            -1232.
Df Residuals:                      505   BIC:                            -1176.
Df Model:                           12                                         
Covariance Type:             nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const               

In [30]:

Y_pred = model.predict(sm.add_constant(X_test))
Y_pred_new = model_new.predict(sm.add_constant(X_test_new))

mse_model = mean_squared_error(Y_test, Y_pred)
mse_model_new = mean_squared_error(Y_test_new, Y_pred_new)

print("Error cuadrático medio del primer modelo:", mse_model)
print("Error cuadrático medio del segundo modelo:", mse_model_new)

if mse_model < mse_model_new:
    print("El primer modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.")
elif mse_model > mse_model_new:
    print("El segundo modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.")
else:
    print("Ambos modelos tienen el mismo error cuadrático medio.")

Error cuadrático medio del primer modelo: 0.0041733683876170775
Error cuadrático medio del segundo modelo: 0.004158914217361775
El segundo modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.


In [151]:
import statsmodels.api as sm

features = ['targeted_productivity', 'smv', 'over_time', 'incentive',
            'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
            'team_1', 'team_2', 'team_3', 'team_4', 'team_5',
            'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 
            'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
            'quarter_Quarter4', 'day_Monday', 'day_Saturday',
            'day_Sunday', 'day_Thursday', 'day_Tuesday', 
            'department_finishing']

X = df_sinWIP[features]
Y = df_sinWIP['actual_productivity']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

X_train = sm.add_constant(X_train)

model = sm.OLS(Y_train, X_train).fit()

#########################################################################

new_features = ['targeted_productivity', 'smv', 'incentive', 'idle_men', 
                'team_1', 'team_2', 'team_3', 'team_4', 'team_5', 'team_6', 
                'team_7', 'team_8', 'team_9', 'team_10', 'team_11',
                'no_of_style_change', 'no_of_workers', 'quarter_Quarter1', 
                'quarter_Quarter2', 'quarter_Quarter3', 'quarter_Quarter4', 
                'department_finishing'
                ]

X_new = df_sinWIP[new_features]
Y_new = df_sinWIP['actual_productivity']

X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(X_new, Y_new, random_state=1)

X_train_new = sm.add_constant(X_train_new)

model_new = sm.OLS(Y_train_new, X_train_new).fit()

################################################################

Y_pred = model.predict(sm.add_constant(X_test))
Y_pred_new = model_new.predict(sm.add_constant(X_test_new))

mse_model = mean_squared_error(Y_test, Y_pred)
mse_model_new = mean_squared_error(Y_test_new, Y_pred_new)

print("Error cuadrático medio del primer modelo:", mse_model)
print("Error cuadrático medio del segundo modelo:", mse_model_new)

if mse_model < mse_model_new:
    print("El primer modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.")
elif mse_model > mse_model_new:
    print("El segundo modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.")
else:
    print("Ambos modelos tienen el mismo error cuadrático medio.")


Error cuadrático medio del primer modelo: 0.025878253902111344
Error cuadrático medio del segundo modelo: 0.025824574734387453
El segundo modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.


In [ ]:
Error cuadrático medio del primer modelo: 0.025878253902111344
Error cuadrático medio del segundo modelo: 0.025997343336882146
El primer modelo tiene un error cuadrático medio más bajo y, por lo tanto, mejor desempeño.